# Introduction to Geovisualisation

This Jupyter notebook demonstrates how objects located at different places at different times can be displayed on a map.

The notebook can be run locally on the computer or in a cloud-based environment such as Google Colab.

## Load necessary modules 

In [ ]:
import pandas as pd
import os
import subprocess


from ipyleaflet import Map, basemaps, WMSLayer, Marker, AntPath, Popup, FullScreenControl
from ipywidgets import HTML, VBox

## Define working directory for Google Colab

In [ ]:
try:
    from google.colab import drive
    in_colab = True
    
except ImportError:
    in_colab = False

if in_colab:
  
  # Define the working directory in google colab.
  working_directory = '/content/drive/My Drive/Colab Notebooks/'

  # Mount personal google drive.
  drive.mount('/content/drive')

## Choose a base map

### Base maps of the module

More information: https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/basemaps.html

In [ ]:
map_center = (46.950, 7.445)
zoom_level = 12
m1 = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=map_center, zoom=zoom_level)

m1

### Base maps from the Federal Office of Topography swisstopo

Examples of layers:
- ch.swisstopo.pixelkarte-farbe
- ch.swisstopo.pixelkarte-grau
- ch.swisstopo.swissimage

In [ ]:
wms_layer = WMSLayer(
    url = 'https://wms.geo.admin.ch/',
    layers='ch.swisstopo.pixelkarte-farbe',
    format='image/png',
    attribution='© swisstopo'
)
m2 = Map(basemap=wms_layer, center=map_center, zoom=zoom_level)

m2

## Import data

We use a CSV file as the basis for displaying objects on the map. The following columns are required for display on the map:
| Name of the column   | Description          |
|----------|----------------------|
| lon | Longitude coordinate in the coordinate system EPSG:4326 (World Geodetic System 1984, WGS 84) |
| lat | Latitude coordinate in the coordinate system EPSG:4326 (World Geodetic System 1984, WGS 84) |
| datetime | The date and time at which the object was located at this time |
| description | Description of the location |
| object_id | Unique key of the object |

In [ ]:


filename_data = 'example_data.csv'
url_data = 'https://raw.githubusercontent.com/history-unibas/economies-of-space-teaching-summer-school-ss2025/refs/heads/main/example_data.csv'

# Save the data in colab.
if in_colab:
    
    if not os.path.exists(working_directory + filename_data):
        # Download the data in working directory.
        subprocess.run(['wget', url_data, '-O', working_directory + filename_data], check=True)
      
    # Update the filepath.
    filename_data = working_directory + filename_data

In [ ]:
# Load the data.
df = pd.read_csv(filename_data, parse_dates=['datetime'])

display(df)

## Show data on the map

In [ ]:
# Shift in degree.
shift = 0.0001

# Create a map for each object.
maps = []
grouped = df.groupby('object_id')
for _, group in grouped:
    
    # Define a map.
    map_center = (group['lat'].mean(), group['lon'].mean())
    m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=map_center, zoom=zoom_level)

    group_sorted = group.sort_values(by='datetime')
    coords_prev = ()
    
    for _, entry in group_sorted.iterrows():
        
        # Add markers with popup to the map.
        marker = Marker(location=(entry['lat'], entry['lon']))
        popup = Popup(location=(entry['lat'], entry['lon']), child=HTML(entry['description']), close_button=True)
        marker.popup = popup        
        m.add_layer(marker)

        coords_current = (entry['lat'], entry['lon'])
        if coords_prev:
            
            # Shift the coordinates.
            if coords_current[0] < coords_prev[0]:
                coords_prev_shifted = (coords_prev[0] + shift, coords_prev[1] + shift)
                coords_current_shifted = (coords_current[0] + shift, coords_current[1] + shift)
            else:
                coords_prev_shifted = (coords_prev[0] - shift, coords_prev[1] - shift)
                coords_current_shifted = (coords_current[0] - shift, coords_current[1] - shift)

            # Create a line between both points.
            arrow_line = AntPath(locations=[coords_prev_shifted, coords_current_shifted],
                                 delay=4000,
                                 weight=4
                                 )
            m.add_layer(arrow_line)
            
        coords_prev = tuple(coords_current)

    maps.append(m)

# Show the maps.
vbox = VBox(maps)
display(vbox)

## Export the map as html file

In [ ]:
filename_map = 'map.html'

# Export the first map as an example.
m3 = maps[0]
m3.add_control(FullScreenControl())

if in_colab:
    m3.save(working_directory + filename_map)

else:
    m3.save(filename_map)